In [5]:
import numpy as np
import pandas as pd
import subprocess
import requests
from time import sleep
from pathlib import Path
import datetime

In [6]:
datetime.datetime.now()

datetime.datetime(2021, 10, 26, 21, 54, 51, 706189)

In [7]:
def generate_intervals(initial, interval_size):
    intervals = []
    intervals.append((0, initial))
    for x in range(interval_size - 1):
        initial_interval_value = intervals[-1][1]
        interval_value = intervals[0][1]

        last_interval_value = initial_interval_value + interval_value
        intervals.append((initial_interval_value + 1, last_interval_value))

    return intervals

In [8]:
def build_table(min_pods, max_pods, initial_lat, interval_lat_size, initial_req, interval_req_size):
    requests_intervals = generate_intervals(initial_req, interval_req_size)

    table = []

    for request_interval in requests_intervals:
        latency_intervals = generate_intervals(initial_lat, interval_lat_size)
        for latency_interval in latency_intervals:
            for pod in range(min_pods, max_pods + 1):
                options = np.zeros(max_pods + 1 - min_pods)
                options = [0 for opt in range(len(options))]

                table.append([pod, latency_interval[0], latency_interval[1], request_interval[0], request_interval[1]] + list(options))

    labels = ['pod', 'initial_latency', 'end_latency', 'initial_request', 'end_request']
    actions = list(np.arange(min_pods, max_pods + 1).astype(np.str_))

    return pd.DataFrame(table, columns=labels+actions)

In [9]:
def find_options(table, pod, latency, request):
    return table[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency) & (table['initial_request'] <= request) & (table['end_request'] >= request)].iloc[:,5::]

In [10]:
def find_best_q_value(table, pod, latency, request):
    print("find_best_q_value")
    options = find_options(table, pod, latency, request)
    
    return int(options.max(axis=1))

In [11]:
def find_best_action(table, pod, latency, request):
    print("find_best_action")
    options = find_options(table, pod, latency, request)
    
    return int(options.idxmax(axis=1))

In [12]:
def update_action_result(table, pod, latency, request, action, result):
    print("update_action_result")
    
    table.loc[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency) & (table['initial_request'] <= request) & (table['end_request'] >= request), str(action)] = result
    print("updating pod", pod, " latency ", latency, " request ", request, " result ", result)
    return table

In [13]:
def get_latency():
    print("get_latency")
    query = 'rate(http_server_request_duration_seconds_sum{path="/test"}[30s])/rate(http_server_request_duration_seconds_count{path="/test"}[30s])'
    result = None

    while(True):
        response = requests.get("http://localhost:9090/api/v1/query?query={query}".format(query = query))
        if response.json()['status'] == 'success':
            result = response.json()['data']['result']
            if result == []:
                result = float(0)
            else:
                result = response.json()['data']['result'][0]['value'][1]
                if result == 'NaN': result = float(0)
            break
        sleep(5)
    
    return round(float(result) * 1000)

In [14]:
def get_request_by_second():
    print("get_request_by_second")    
    query = 'increase(http_server_requests_total{path="/test"}[30s])'
    result = None

    while(True):
        response = requests.get("http://localhost:9090/api/v1/query?query={query}".format(query = query))
        if response.json()['status'] == 'success':
            result = response.json()['data']['result']
            if result == []:
                result = float(0)
            else:
                result = response.json()['data']['result'][0]['value'][1]
                if result == 'NaN': result = float(0)
            break
        sleep(5)
        
    req = round(float(result))
    if req > 54:
        req = 54
    
    return req

In [15]:
def get_pods():
    print("get_pods")    
    # TODO comando pega quando esta como Terminating
    bashCommand = "kubectl get pods --field-selector=status.phase=Running"
    
    error = True
    output = None
    
    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    
    return len(output.decode("utf-8").split('\n')[1:][:-1])  
    

In [16]:
def set_pods(new_pods):
    print("set_pods")
    bashCommand = "kubectl scale deployment.v1.apps/phpa-web-app-deployment --replicas={pods}".format(pods = new_pods)

    error = True
    output = None

    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()

In [17]:
def run(epochs, table, min_pods, max_pods):
    ideal_latency = 500
    alpha = 0.1
    gamma = 0.6
    epsilon = 0.1
    
    history_file = Path("history.csv")
    if history_file.is_file():
        history = pd.read_csv("history.csv")
    else:
        history = pd.DataFrame([], columns=['epoch','pod', 'latency', 'requests', 'action', 'new_latency', 'new_requests', 'reward', 'q_value', 'random'])      
    
    latency = get_latency()
    pods = get_pods()
    requests = get_request_by_second()

    for x in range(epochs):
        print("\n--- Exec ", x,"\n\n latency ", latency, "\n pods ", pods, "\n requests ", requests)
        is_random = None

        if np.random.uniform(0, 1) > epsilon:
            action = find_best_action(table, pods, latency, requests)
            is_random = False
            print("choose action ", action)
        else:
            action = np.random.randint(min_pods, max_pods + 1)
            is_random = True
            print("random ", action)

        set_pods(action)
        sleep(45)

        new_latency = get_latency()
        new_requests = get_request_by_second()
        reward = get_reward(ideal_latency, pods, action, latency, new_latency)
        print(" action ", action, "\n new_latency ", new_latency, "\n new_requests ", new_requests, "\n reward ", reward)

        new_value = new_q_value(table, pods, latency, requests, action, new_latency, new_requests, reward, alpha, gamma)
        print("*** new value: ", new_value, " ***")
        
        table = update_action_result(table, pods, latency, requests, action, new_value)
        values = {'epoch': x,'pod': pods, 'latency': latency, 'requests': requests, 'action': action, 'new_latency': new_latency, 'new_requests': new_requests, 'reward': reward, 'q_value' : new_value, 'random': is_random}
        history = history.append(values, ignore_index=True)
        # setting new state
        latency = new_latency
        pods = action
        requests = new_requests
    
    table.to_csv('table.csv', index=False)
    history.to_csv('history.csv', index=False)
    print("acabou: ", datetime.datetime.now())

In [18]:
def new_q_value(table, pods, latency, requests, action, new_latency, new_requests, reward, alpha, gamma):
    old_value = find_best_q_value(table, pods, latency, requests)
    next_max = find_best_q_value(table, action, new_latency, new_requests)

    return (1 - alpha) * old_value + alpha * (reward + gamma * next_max)

In [19]:
def is_ideal_latency(ideal_latency, latency):
    print("is_ideal_latency")
    return latency <= ideal_latency

In [20]:
def obtained_result(new_value, old_value):
    print("obtained_result")    
    if new_value == old_value:
        return 'kept'
    elif new_value > old_value:
        return 'increased'
    else:
        return 'decreased'

In [21]:
def reward_within_ideal(pod_state, latency_state, new_latency_is_ideal):
    print("reward_within_ideal")
    if not new_latency_is_ideal:
        return -20 # qualquer ação que tomou resultou em levar a latencia para acima do desejado
    
    if pod_state == 'kept': return 10 # não tomou nenhuma ação, mas manteve a latencia ideal
    elif pod_state == 'increased':
        return {
            'kept': -10, # aumentou pods e a latencia se manteve - pode ter aumentado as requisições
            'increased': 0, # aumentou pods e a latencia aumentou - pode ter aumentado as requisições
            'decreased': -20 # ja estava na latencia ideal e aumentou pods desnecessáriamente
        }.get(latency_state)
    else: # pod_state == 'decreased'
        return {
            'kept': 20, # diminiu pods e a latencia se manteve
            'increased': 20, # diminiu pods, latencia aumentou mas segue estando dentro do desejado: ótimo cenário
            'decreased': 20 # diminiu pods, diminiu latencia - pode ter diminuido as requisições
        }.get(latency_state)

In [22]:
def reward_out_ideal(pod_state, latency_state, new_latency_is_ideal):
    print("reward_out_ideal")    
    if new_latency_is_ideal:
        return 20 # qualquer ação tomada resultou em levar a latencia para dentro do desejado
    
    if pod_state == 'kept':
        return {
            'kept': -20, # não tomou nenhuma ação
            'increased': -20, # não tomou nenhuma ação e aumentou a latencua
            'decreased': -10 # não tomou nenhuma ação mas diminiu a latencia - pode ter diminuido requisições
        }.get(latency_state)
    elif pod_state == 'increased':
        return {
            'kept': 0, # aumentou pods e manteve a latencia - podem ter aumentado as requisições
            'increased': 0, # aumentou pods e aumentou latencia - podem ter aumentado as requisições
            'decreased': 10 # aumentou pods e diminiu a latencia
        }.get(latency_state)
    else: # pod_state == 'decreased'
        return {
            'kept': -10, # diminiu pods e a latencia se manteve
            'increased': -20, # diminiu pods, latencia aumentou
            'decreased': 0 # diminiu pods, diminiu latencia - pode ter diminuido as requisições
        }.get(latency_state)

In [23]:
def get_reward(ideal_latency, old_pods, new_pods, old_latency, new_latency):
    print("get_reward")    
    pod_state = obtained_result(new_pods, old_pods)
    latency_state = obtained_result(new_latency, old_latency)

    old_latency_is_ideal = is_ideal_latency(ideal_latency, old_latency)
    new_latency_is_ideal = is_ideal_latency(ideal_latency, new_latency)

    reward = 0
    if old_latency_is_ideal:
        reward = reward_within_ideal(pod_state, latency_state, new_latency_is_ideal)
    else:
        reward = reward_out_ideal(pod_state, latency_state, new_latency_is_ideal)

    return reward

In [32]:
epochs = 2100
min_pods = 1
max_pods = 3
initial_latency = 500
interval_size = 4
initial_request = 9
interval_request = 66

In [33]:
interval_request

66

In [41]:
# table = build_table(min_pods, max_pods, initial_latency, interval_size, initial_request, interval_request)
# table

table_file = Path("table.csv")
if table_file.is_file():
    table = pd.read_csv("table.csv")
else:
    table = build_table(min_pods, max_pods, initial_latency, interval_size, initial_request, interval_request)

table

,pod,initial_latency,end_latency,initial_request,end_request,1,2,3
0,1,0,500,0,9,0.00,0.00,0.00
1,2,0,500,0,9,-1.16,1.00,-1.82
2,3,0,500,0,9,-1.40,-1.28,2.92
3,1,501,1000,0,9,0.00,0.00,0.00
4,2,501,1000,0,9,-1.76,2.90,0.00
...,...,...,...,...,...,...,...,...
67,2,1001,1500,46,54,0.00,0.00,0.00
68,3,1001,1500,46,54,0.00,0.00,0.00
69,1,1501,2000,46,54,0.92,8.74,6.22
70,2,1501,2000,46,54,0.00,0.00,0.00


In [42]:
print("Start at: ", datetime.datetime.now())

Start at:  2021-10-26 23:00:15.958118


In [43]:
run(epochs, table, min_pods, max_pods)

get_latency
get_pods
get_request_by_second

--- Exec  0 

 latency  275 
 pods  1 
 requests  6
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  207 
 new_requests  4 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  1.0  ***
update_action_result
updating pod 1  latency  275  request  6  result  1.0

--- Exec  1 

 latency  207 
 pods  1 
 requests  4
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  309 
 new_requests  8 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  1.96  ***
update_action_result
updating pod 1  latency  207  request  4  result  1.96

--- Exec  2 

 latency  309 
 pods  1 
 requests  8
find_best_action
choose action  1
set_pods
get_latency

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  2 
 new_latency  208 
 new_requests  2 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  1.96  ***
update_action_result
updating pod 2  latency  282  request  6  result  1.96

--- Exec  20 

 latency  208 
 pods  2 
 requests  2
find_best_action
choose action  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  2 
 new_latency  209 
 new_requests  4 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  1.96  ***
update_action_result
updating pod 2  latency  208  request  2  result  1.96

--- Exec  21 

 latency  209 
 pods  2 
 requests  4
find_best_action
choose action  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  2 
 ne

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  311 
 new_requests  12 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  1.96  ***
update_action_result
updating pod 1  latency  312  request  8  result  1.96

--- Exec  40 

 latency  311 
 pods  1 
 requests  12
random  3
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  3 
 new_latency  207 
 new_requests  2 
 reward  -20
find_best_q_value
find_best_q_value
*** new value:  -0.9800000000000001  ***
update_action_result
updating pod 1  latency  311  request  12  result  -0.9800000000000001

--- Exec  41 

 latency  207 
 pods  3 
 requests  2
find_best_action
choose action  3
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 acti

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  207 
 new_requests  4 
 reward  20
find_best_q_value
find_best_q_value
*** new value:  3.8600000000000003  ***
update_action_result
updating pod 3  latency  0  request  0  result  3.8600000000000003

--- Exec  60 

 latency  207 
 pods  1 
 requests  4
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  290 
 new_requests  12 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  1.96  ***
update_action_result
updating pod 1  latency  207  request  4  result  1.96

--- Exec  61 

 latency  290 
 pods  1 
 requests  12
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  2 
 new_latency  0 
 new_requests  0 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  3.8800000000000003  ***
update_action_result
updating pod 2  latency  275  request  6  result  3.8800000000000003

--- Exec  80 

 latency  0 
 pods  2 
 requests  0
random  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  308 
 new_requests  8 
 reward  20
find_best_q_value
find_best_q_value
*** new value:  4.76  ***
update_action_result
updating pod 2  latency  0  request  0  result  4.76

--- Exec  81 

 latency  308 
 pods  1 
 requests  8
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_

...

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  401 
 new_requests  18 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  1.96  ***
update_action_result
updating pod 1  latency  468  request  14  result  1.96

--- Exec  2024 

 latency  401 
 pods  1 
 requests  18
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  471 
 new_requests  14 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  1.96  ***
update_action_result
updating pod 1  latency  401  request  18  result  1.96

--- Exec  2025 

 latency  471 
 pods  1 
 requests  14
random  3
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  3 
 new_latency  312

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  411 
 new_requests  18 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  0.16000000000000003  ***
update_action_result
updating pod 1  latency  454  request  20  result  0.16000000000000003

--- Exec  2043 

 latency  411 
 pods  1 
 requests  18
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  454 
 new_requests  20 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  1.9  ***
update_action_result
updating pod 1  latency  411  request  18  result  1.9

--- Exec  2044 

 latency  454 
 pods  1 
 requests  20
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_laten

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  2 
 new_latency  307 
 new_requests  8 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  4.0  ***
update_action_result
updating pod 2  latency  346  request  12  result  4.0

--- Exec  2062 

 latency  307 
 pods  2 
 requests  8
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  414 
 new_requests  18 
 reward  20
find_best_q_value
find_best_q_value
*** new value:  6.5  ***
update_action_result
updating pod 2  latency  307  request  8  result  6.5

--- Exec  2063 

 latency  414 
 pods  1 
 requests  18
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 


get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  411 
 new_requests  12 
 reward  20
find_best_q_value
find_best_q_value
*** new value:  11.0  ***
update_action_result
updating pod 3  latency  253  request  8  result  11.0

--- Exec  2081 

 latency  411 
 pods  1 
 requests  12
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  487 
 new_requests  22 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  1.06  ***
update_action_result
updating pod 1  latency  411  request  12  result  1.06

--- Exec  2082 

 latency  487 
 pods  1 
 requests  22
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 actio

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  470 
 new_requests  14 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  1.96  ***
update_action_result
updating pod 1  latency  488  request  22  result  1.96
acabou:  2021-10-28 02:27:43.129285


In [46]:
table

,pod,initial_latency,end_latency,initial_request,end_request,1,2,3
0,1,0,500,0,9,1.96,-0.68,-0.80
1,2,0,500,0,9,6.50,1.96,-1.10
2,3,0,500,0,9,11.00,-3.26,-2.96
3,1,501,1000,0,9,0.00,0.00,0.00
4,2,501,1000,0,9,-1.76,4.34,0.00
5,3,501,1000,0,9,4.96,-0.44,7.16
6,1,1001,1500,0,9,0.00,0.00,0.00
7,2,1001,1500,0,9,0.00,0.00,0.00
8,3,1001,1500,0,9,0.00,0.00,0.00
9,1,1501,2000,0,9,0.00,0.00,0.00


In [45]:
pd.set_option('display.max_rows', None)

In [53]:
build_table(min_pods, max_pods, initial_latency, interval_size, initial_request, interval_request)

,pod,initial_latency,end_latency,initial_request,end_request,1,2,3
0,1,0,500,0,9,0,0,0
1,2,0,500,0,9,0,0,0
2,3,0,500,0,9,0,0,0
3,1,501,1000,0,9,0,0,0
4,2,501,1000,0,9,0,0,0
...,...,...,...,...,...,...,...,...
67,2,1001,1500,46,54,0,0,0
68,3,1001,1500,46,54,0,0,0
69,1,1501,2000,46,54,0,0,0
70,2,1501,2000,46,54,0,0,0


- testar quanto tempo leva pra subir um pod. Verificar se é possível diminuir o tempo que esta em 100s para tvz 30s ja que tera dado tempo de rodar ao menos uma leitura do prometheus (que esta a cada 15s)
- alterar q table para colocar requisições de 9 em 9 ao inves de 18 em 18

In [55]:
def coisa(epochs, pods):
    without_phpa = pd.DataFrame([], columns=['epoch','pod', 'latency', 'requests'])      

    for x in range(epochs):
        latency = get_latency()
        pods = get_pods()
        requests = get_request_by_second()
        
        print("\n--- Exec ", x,"\n\n latency ", latency, "\n pods ", pods, "\n requests ", requests)
        
        values = {'epoch': x,'pod': pods, 'latency': latency, 'requests': requests}
        without_phpa = without_phpa.append(values, ignore_index=True)
        
        sleep(45)

    without_phpa.to_csv('without_phpa.csv', index=False)
    print("acabou: ", datetime.datetime.now())

In [ ]:
coisa(2100, 2)

get_latency
get_pods
get_request_by_second

--- Exec  0 

 latency  276 
 pods  1 
 requests  8
